<a href="https://colab.research.google.com/github/anandhc6/Assignment-2/blob/main/Assignment_2_Part_A_Sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Downloading the dataset as zip folder

!gdown https://storage.googleapis.com/wandb_datasets/nature_12K.zip

In [ ]:
# Unzipping the dataset

!unzip -q nature_12K.zip

In [ ]:
#installing wandb

!pip install wandb --upgrade

     |████████████████████████████████| 1.7 MB 8.4 MB/s 
     |████████████████████████████████| 144 kB 62.3 MB/s 
     |████████████████████████████████| 181 kB 77.9 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=77f8916371e0f1d3fed3187e03be13a0e22b79a57c3cf67ad966b1954feeb9ef
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
# Importing necessary packages

import os
import glob
import wandb
import pathlib
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from wandb.keras import WandbCallback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization


In [ ]:
#Data preperation

def datagen(batch_size, augment_data):

  Data_dir=pathlib.Path('inaturalist_12K')
  # augment_data=False
  train_path = os.path.join(Data_dir, "train")
  test_path = os.path.join(Data_dir, "val")

  if augment_data:
    train_rawdata = ImageDataGenerator(rescale=1./255,
                                      rotation_range=90,
                                      zoom_range=0.2,
                                      shear_range=0.2,
                                      validation_split=0.1,
                                      horizontal_flip=True)
    test_rawdata = ImageDataGenerator(rescale=1./255)

  else:
    train_rawdata = ImageDataGenerator(rescale=1./255, validation_split=0.1)
    test_rawdata = ImageDataGenerator(rescale=1./255)

  train_data = train_rawdata.flow_from_directory(
      train_path, 
      target_size=(256, 256), 
      batch_size=batch_size, 
      subset="training",
      color_mode="rgb",
      class_mode='categorical',
      shuffle=True,
      seed=42
      )
  val_data = train_rawdata.flow_from_directory(
      train_path, 
      target_size=(256, 256), 
      batch_size=batch_size, 
      subset="validation",
      color_mode="rgb",
      class_mode='categorical',
      shuffle=True,
      seed=42
      )
  test_data = test_rawdata.flow_from_directory(
      test_path, 
      target_size=(256, 256), 
      batch_size=batch_size,
      color_mode="rgb",
      class_mode='categorical',
      shuffle=True,
      seed=42
      )
  return  train_data, val_data, test_data

In [ ]:
def CNN_model(activation_func_convolution, activation_func_dense, no_of_filters, filter_convol_size, filter_pool_size, batch_normalisation, dense_layer, dropout,image_size):

  no_of_classes =10
  filter_sizes =[]
  filter_sizes.append(no_of_filters)
  for layer_num in range(1,3):
      filter_sizes.append(no_of_filters*(2**layer_num))
      filter_sizes.append(no_of_filters*(2**layer_num))
  #no_of_fil=no_of_filters/4
  # for layer_num in range(0,5):
  #     #filter_sizes.append(no_of_fil*)
  #     filter_sizes.append(no_of_fil*(2**layer_num))

  # convolution layers
  model = Sequential()
  #print("Check 1")
  model.add(Conv2D(filter_sizes[0], kernel_size=(filter_convol_size,filter_convol_size), input_shape=(image_size, image_size, 3), data_format="channels_last"))
  if batch_normalisation:
      model.add(BatchNormalization())
  model.add(Activation(activation_func_convolution[0]))
  model.add(MaxPooling2D(pool_size=filter_pool_size[0] ))
  #print("Check 2")

  for i in range(1, 5):
      model.add(Conv2D(filter_sizes[i], kernel_size=(filter_convol_size,filter_convol_size)))
      if batch_normalisation:
          model.add(BatchNormalization())
      model.add(Activation(activation_func_convolution[i]))
      model.add(MaxPooling2D(pool_size=filter_pool_size[i]))
  
  # Converting the feature map into a column vector and regularisation
  model.add(Flatten()) 
  model.add(Dense(dense_layer, activation=activation_func_dense))
  model.add(Dropout(dropout)) 
  model.add(Dense(no_of_classes, activation="softmax")) 

  return model

In [ ]:
#Function called by sweep

def model_train():
    
    config_defaults = {
        "augment_data": True,
        "batch_size": 64, 
        "batch_normalisation": True,
        "dropout": 0,
        "no_of_filters": [64],
        "dense_layer": 128,
        "filter_convol_size": [3]
    }

    # Constant hyperparameters
    filter_pool_size = [(2, 2), (2, 2), (2, 2), (2, 2), (2, 2)]
    image_size = 256
    activation_func_convolution = ["relu", "relu", "relu", "relu", "relu"]
    activation_func_dense = "relu"
    
    
    # Initializing new wandb run and saving hyperparameters,inputs
    wandb.init(config=config_defaults)
    config = wandb.config

    # Values obtained from wandb config(Sweep)
    no_of_filters = config.no_of_filters
    augment_data = config.augment_data
    batch_size = config.batch_size
    dense_layer = config.dense_layer
    filter_convol_size = config.filter_convol_size
    batch_normalisation = config.batch_normalisation
    dropout = config.dropout
    

    # Filters calculation for 5 layers
    filter_name=[]
    #num_fil=no_of_filters/4
    filter_name.append(no_of_filters)
    for layer_num in range(1,3):
      # filter_sizes.append(no_of_filters*(2**layer_num))
      # filter_sizes.append(no_of_filters*(2**layer_num))
        filter_name.append(no_of_filters*(2**layer_num))
        filter_name.append(no_of_filters*(2**layer_num))
    
    # Run name with hyperparameters
    run_name = "fil_{}_aug_{}_bs_{}_bn_{}_drop_{}_fc_{}_fil_{}_shape_{}".format(filter_name,augment_data, batch_size, batch_normalisation, dropout, dense_layer, no_of_filters, filter_convol_size)
    print(run_name)

    # Data preperation
    train_data, val_data, test_data =datagen(batch_size, augment_data)
    print("Data recieved")

    # Model creation
    model = CNN_model(activation_func_convolution,activation_func_dense , no_of_filters, filter_convol_size, filter_pool_size, batch_normalisation, dense_layer, dropout ,image_size)
    print(model.count_params())
    print("Model Training done")

    model.compile(optimizer=Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

    # Early Stopping 
    earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
    
    train_step_size = train_data.n//train_data.batch_size
    print(train_step_size)
    val_step_size = val_data.n//val_data.batch_size
    print(val_step_size)

    model_det = model.fit(train_data,
          steps_per_epoch = train_step_size,
          validation_data = val_data,
          validation_steps = val_step_size,
          epochs=20, 
          callbacks=[WandbCallback(data_type="image", generator=val_data), earlyStopping, mc],
          verbose=2)

    
    wandb.run.name = run_name
    wandb.run.save()
    wandb.run.finish()
    return model_det

In [ ]:
# Sweep configuration
sweep_config = {
  "name": "Sweep_trial_DL_assignment_2",
  "metric": {
      "name":"val_accuracy",
      "goal": "maximize"
  },
  "method": "bayes",
  "parameters": {
        "augment_data": {
            "values": [True,False]
        },
        "batch_size": {
            "values": [32,64,128]
        },
        "batch_normalisation": {
            "values": [True,False]
        },
        "dropout": {
            "values": [0,0.3,0.2,0.15]
        },
        "no_of_filters": {
            "values": [32, 64]
        },
        "dense_layer": {
            "values": [64,128]
        },
        "filter_convol_size": {
            "values": [3,5,7],
                    
        }
    }
}

# Generates a sweep id
# sweep_id = wandb.sweep(sweep_config, project = 'DL-Assignment2-PartA-5April-2')
# wandb.agent(sweep_id, function=CNN_train)

# sweep_id = wandb.sweep(sweep_config, project="DL_assignment_2")
# wandb.agent(sweep_id, model_train , count=2)

sweep_id ="dhvdsf58"
wandb.agent(sweep_id, model_train, entity="anandh", project = "DL_assignment_2", count = 10)


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Agent Starting Run: lmzyrn4v with config:
wandb: 	augment_data: True
wandb: 	batch_normalisation: False
wandb: 	batch_size: 128
wandb: 	dense_layer: 128
wandb: 	dropout: 0.15
wandb: 	filter_convol_size: 5
wandb: 	no_of_filters: 64
wandb: Currently logged in as: anandh (use `wandb login --relogin` to force relogin)


fil_[64, 128, 128, 256, 256]_aug_True_bs_128_bn_False_drop_0.15_fc_128_fil_64_shape_5
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Data Vaangiyachi
Check 1


...

fil_[64, 128, 128, 256, 256]_aug_False_bs_128_bn_True_drop_0.15_fc_128_fil_64_shape_5
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Data Vaangiyachi
Check 1
Check 2
3606666
Training done
70
7
Epoch 1/20
70/70 - 109s - loss: 2.1414 - accuracy: 0.2497 - val_loss: 2.3863 - val_accuracy: 0.1038 - _timestamp: 1648414528.0000 - _runtime: 116.0000 - 109s/epoch - 2s/step
Epoch 2/20
70/70 - 105s - loss: 1.8903 - accuracy: 0.3371 - val_loss: 2.4762 - val_accuracy: 0.1049 - _timestamp: 1648414666.0000 - _runtime: 254.0000 - 105s/epoch - 1s/step
Epoch 3/20
70/70 - 105s - loss: 1.7544 - accuracy: 0.3883 - val_loss: 2.4374 - val_accuracy: 0.1484 - _timestamp: 1648414771.0000 - _runtime: 359.0000 - 105s/epoch - 1s/step
Epoch 4/20
70/70 - 105s - loss: 1.5799 - accuracy: 0.4592 - val_loss: 2.4613 - val_accuracy: 0.1194 - _timestamp: 1648414876.0000 - _runtime: 464.0000 - 105s/epoch - 1s/step
Epoch 5/20
70/70 - 105s - loss

accuracy,▁▃▄▅▆█
epoch,▁▂▄▅▇█
loss,█▆▅▄▂▁
val_accuracy,▁▁▅▃▅█
val_loss,▁▅▃▄▄█
accuracy,0.59558
best_epoch,0
best_val_loss,2.38629
epoch,5
loss,1.21239
val_accuracy,0.17522


wandb: Agent Starting Run: gt62q56w with config:
wandb: 	augment_data: False
wandb: 	batch_normalisation: True
wandb: 	batch_size: 128
wandb: 	dense_layer: 128
wandb: 	dropout: 0.15
wandb: 	filter_convol_size: 3
wandb: 	no_of_filters: 64


fil_[64, 128, 128, 256, 256]_aug_False_bs_128_bn_True_drop_0.15_fc_128_fil_64_shape_3
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Data Vaangiyachi
Check 1
Check 2
2292874
Training done
70
7
Epoch 1/20
70/70 - 107s - loss: 2.1706 - accuracy: 0.2316 - val_loss: 2.3577 - val_accuracy: 0.1004 - _timestamp: 1648415239.0000 - _runtime: 114.0000 - 107s/epoch - 2s/step
Epoch 2/20
70/70 - 105s - loss: 1.9270 - accuracy: 0.3250 - val_loss: 2.4696 - val_accuracy: 0.1049 - _timestamp: 1648415344.0000 - _runtime: 219.0000 - 105s/epoch - 1s/step
Epoch 3/20
70/70 - 106s - loss: 1.8087 - accuracy: 0.3744 - val_loss: 2.5591 - val_accuracy: 0.1116 - _timestamp: 1648415450.0000 - _runtime: 325.0000 - 106s/epoch - 2s/step
Epoch 4/20
70/70 - 105s - loss: 1.6553 - accuracy: 0.4222 - val_loss: 2.5123 - val_accuracy: 0.1362 - _timestamp: 1648415555.0000 - _runtime: 430.0000 - 105s/epoch - 1s/step
Epoch 5/20
70/70 - 105s - loss

accuracy,▁▂▂▃▃▄▅▅▆▇▇▇██
epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
loss,█▇▇▆▆▅▄▄▃▃▂▂▁▁
val_accuracy,▁▁▁▂▂▄▅▇▇▇████
val_loss,▆▇█▇█▅▆▁▁▂▂▂▂▂
accuracy,0.92809
best_epoch,8
best_val_loss,1.98655
epoch,13
loss,0.35716
val_accuracy,0.34933
